In [1]:
from keras.layers import Lambda, Input, Dense, Conv2D, Flatten
from keras.models import Model, Sequential
#from keras.datasets import mnist
from keras.losses import mse, binary_crossentropy
from keras.utils import plot_model
from keras import backend as K
from keras import metrics
from keras.utils import plot_model

import numpy as np
import matplotlib.pyplot as plt
import argparse
import os
import datetime as dt
import time
import glob
from IPython import display
import tensorflow as tf
#tfe = tf.contrib.eager
#tf.enable_eager_execution()


import datetime as dt

Using TensorFlow backend.


In [ ]:
def plot_autoencoder_outputs(encoder, n, dims):
    #decoded_imgs = enconder.predict(x_test)
    x_test_encoded = vae.predict(x_test, batch_size = batch_size)
    # number of example digits to show
    n = 5
    plt.figure(figsize=(20, 9))
    for i in range(n):
        # plot original image
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(x_test[i].reshape(*dims))
        #plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        if i == n/2:
            ax.set_title('Original Images')

        # plot reconstruction 
        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(x_test_encoded[i].reshape(*dims))
        #plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        if i == n/2:
            ax.set_title('Reconstructed Images')
    plt.show()

def plot_loss(history):
    historydf = pd.DataFrame(history.history, index=history.epoch)
    plt.figure(figsize=(8, 6))
    historydf.plot(ylim=(0, historydf.values.max()))
    plt.title('Loss: %.3f' % history.history['loss'][-1])
    
def plot_compare_histories(history_list, name_list, plot_accuracy=True):
    dflist = []
    min_epoch = len(history_list[0].epoch)
    losses = []
    for history in history_list:
        h = {key: val for key, val in history.history.items() if not key.startswith('val_')}
        dflist.append(pd.DataFrame(h, index=history.epoch))
        min_epoch = min(min_epoch, len(history.epoch))
        losses.append(h['loss'][-1])

    historydf = pd.concat(dflist, axis=1)

    metrics = dflist[0].columns
    idx = pd.MultiIndex.from_product([name_list, metrics], names=['model', 'metric'])
    historydf.columns = idx
    
    plt.figure(figsize=(6, 8))

    ax = plt.subplot(211)
    historydf.xs('loss', axis=1, level='metric').plot(ylim=(0,1), ax=ax)
    plt.title("Training Loss: " + ' vs '.join([str(round(x, 3)) for x in losses]))
    
    if plot_accuracy:
        ax = plt.subplot(212)
        historydf.xs('acc', axis=1, level='metric').plot(ylim=(0,1), ax=ax)
        plt.title("Accuracy")
        plt.xlabel("Epochs")
    
    plt.xlim(0, min_epoch-1)
    plt.tight_layout()

In [2]:
# Import whole N-MNIST Dataset
def load_NMNIST(path):
    """
    """
    xs_train = []
    ys_train = []
    xs_test = []
    ys_test = []

    for class_index in range(0, 10):
        for (root, dirs, dat_files) in os.walk('{0}/n_Train_3/{1}'.format(path, str(class_index))):
            for file in dat_files:
                single_X = np.fromfile('{0}/n_Train_3/{1}/{2}'.format(path, str(class_index), file), dtype=np.int32)
                xs_train.append(single_X)
                ys_train.append(class_index)

        for (root, dirs, dat_files) in os.walk('{0}/n_Test_3/{1}'.format(path, str(class_index))):
            for file in dat_files:
                xs_test.append(np.fromfile('{0}/n_Test_3/{1}/{2}'.format(path, str(class_index), file), dtype=np.int32))
                ys_test.append(class_index)

    Xtr = np.array(xs_train)
    Xtr_reshaped = Xtr.reshape(60000, 34, 34, 2)
    Ytr = np.array(ys_train)
    Xte = np.array(xs_test)
    Xte_reshaped = Xte.reshape(10000, 34, 34, 2)
    Yte = np.array(ys_test)

    return Xtr_reshaped, Ytr, Xte_reshaped, Yte

In [3]:
dataset_class_path = '/Users/ching/Desktop/Project/autoencoder_nmnist/NMNIST/datasets'
x_train_1, y_train, x_test_1, y_test = load_NMNIST(dataset_class_path)

# As a sanity check, we print out the size of the training and test data.
print('Training data shape: ', x_train_1.shape)
print('Training labels shape: ', y_train.shape)
print('Test data shape: ', x_test_1.shape)
print('Test labels shape: ', y_test.shape)

# 34 x 34 x 2

Training data shape:  (60000, 34, 34, 2)
Training labels shape:  (60000,)
Test data shape:  (10000, 34, 34, 2)
Test labels shape:  (10000,)


In [4]:
# Flatten the dataset from range -1 to 1. Since the maxima and minima lies btw 15 so we divide it by 15.
x_train = (x_train_1.astype('float32'))
x_test = (x_test_1.astype('float32'))


In [5]:
x_train[x_train!=0]+=30
x_test[x_test!=0]+=30
#print(x_train.shape)
#print(max(x_train[0])) #Test the maximum value after flatten


In [6]:
#x_train = x_train/45
x_train[:] = [x / 45 for x in x_train]
#x_test = x_test/45
x_test[:] = [x / 45 for x in x_test]

In [ ]:
TRAIN_BUF = 60000
BATCH_SIZE = 80

TEST_BUF = 10000

train_dataset = tf.data.Dataset.from_tensor_slices(x_train).shuffle(TRAIN_BUF).batch(BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices(x_test).shuffle(TEST_BUF).batch(BATCH_SIZE)


In [ ]:
epsilon_std=0.01

# z = z_mean + sqrt(var)*eps
#returns random value of z with the gives mean and variance
def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0.0, stddev=epsilon_std)
    return z_mean + K.exp(z_log_sigma/2) * epsilon

In [ ]:
#Defining number of kernels, strides and pooling sizes per layer

kernel_1 = 8
kernel_2 = 16
kernel_3 = 32
# kernel_4 = 64
# kernel_5 = 128
# kernel_6 = 256
# kernel_7 = 512

stride_1 = (3, 3)
stride_2 = (2, 2)
stride_3 = (2, 2)
# stride_4 = (2, 2)
# stride_5 = (2, 2)
# stride_6 = (2, 2)
# stride_7 = (2, 2)

pool_1 = (2, 2)
pool_2 = (2, 2)
pool_3 = (2, 2)
# pool_4 = (2, 2)
# pool_5 = (2, 2)
# pool_6 = (2, 2)
# pool_7 = (2, 2)

In [ ]:
#Defining encoder

inputs = Input(shape = x_train_1[0].shape)

x = Conv2D(kernel_1, stride_1, activation='relu', padding='same')(inputs)
x = MaxPooling2D(pool_1, padding='same')(x)

x = Conv2D(kernel_2, stride_2, activation='relu', padding='same')(x)
x = MaxPooling2D(pool_2, padding='same')(x)

x = Conv2D(kernel_3, stride_3, activation='relu', padding='same')(x)
encoded = MaxPooling2D(pool_3, padding='same')(x)

shape = K.int_shape(x)

x = Flatten()(x)

#Adding 1 hidden layer
x = Dense(16, activation='relu')(x)

z_mean = Dense(latent_dim, name='z_mean')(x)
z_log_var = Dense(latent_dim, name='z_log_var')(x)
z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

In [ ]:
encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
encoder.summary()
plot_model(encoder, to_file='vae_cnn_encoder.png', show_shapes=True)


In [ ]:
#Defining Decoder
latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
x = Dense(shape[1] * shape[2] * shape[3], activation='relu')(latent_inputs)
x = Reshape((shape[1], shape[2], shape[3]))(x)


x = Conv2DTranspose(kernel_3, stride_3, activation='relu', padding='same')(x)
x = UpSampling2D(pool_3, padding='same')(x)

x = Conv2DTranspose(kernel_2, stride_2, activation='relu', padding='same')(x)
x = UpSampling2D(pool_2, padding='same')(x)

x = Conv2DTranspose(kernel_1, stride_1, activation='relu', padding='same')(x)
x = UpSampling2D(pool_1, padding='same')(x)

outputs = Conv2DTranspose(filters=1,
                          kernel_size=3,
                          activation='sigmoid',
                          padding='same',
                          name='decoder_output')(x)

In [ ]:
decoder = Model(latent_inputs, outputs, name='decoder')
decoder.summary()
plot_model(decoder, to_file='vae_cnn_decoder.png', show_shapes=True)

In [ ]:
# instantiate VAE model
outputs = decoder(encoder(inputs)[2])
vae = Model(inputs, outputs, name='vae_mlp')
vae_test=Model(inputs, outputs, name='vae_test')

In [ ]:
# Compute VAE loss
xent_loss = 2312 * binary_crossentropy(K.flatten(inputs), K.flatten(outputs))
#xent_loss = binary_crossentropy(inputs, outputs)

kl_loss = -0.5*K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
vae_loss =abs(K.mean(xent_loss + kl_loss))

vae.add_loss(vae_loss)
vae.compile(optimizer='adam')
vae_test.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])
vae.summary()
vae_test.summary()

In [ ]:
start_time = dt.datetime.now()
print('Start learning at {}'.format(str(start_time)))

vae.fit(x_train,
        shuffle=True,
        epochs=epochs,
        batch_size=batch_size,
        validation_split= 0.2)

end_time = dt.datetime.now() 
print('Stop learning {}'.format(str(end_time)))
elapsed_time= end_time - start_time
print('Elapsed learning {}'.format(str(elapsed_time)))

In [ ]:
models = (encoder, decoder)
data = (x_test, y_test)

In [ ]:
def plot_results(models,
                 data,
                 batch_size=100,
                 model_name="vae_mnist"):
    """Plots labels and MNIST digits as function of 2-dim latent vector
    # Arguments:
        models (tuple): encoder and decoder models
        data (tuple): test data and label
        batch_size (int): prediction batch size
        model_name (string): which model is using this function
    """

    encoder, decoder = models
    x_test, y_test = data
    os.makedirs(model_name, exist_ok=True)

    filename = os.path.join(model_name, "vae_mean.png")
    # display a 2D plot of the digit classes in the latent space
    z_mean, _, _ = encoder.predict(x_test,
                                   batch_size=batch_size)
    print(z_mean)
    plt.figure(figsize=(12, 10))
    plt.scatter(z_mean[:, 0], z_mean[:, 1], c=y_test)
    plt.colorbar()
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.savefig(filename)
    plt.show()

    filename = os.path.join(model_name, "digits_over_latent.png")
    # display a 30x30 2D manifold of digits
    n = 30
    digit_size = 28
    figure = np.zeros((digit_size * n, digit_size * n))
    # linearly spaced coordinates corresponding to the 2D plot
    # of digit classes in the latent space
    grid_x = np.linspace(-4, 4, n)
    grid_y = np.linspace(-4, 4, n)[::-1]

In [ ]:
plot_results(models,
                 data,
                 batch_size=batch_size,
                 model_name="vae_mlp")

In [ ]:
def plot_autoencoder_outputs(encoder, n, dims):
    #decoded_imgs = enconder.predict(x_test)
    #x_test_encoded = vae.predict(x_test, batch_size = batch_size)
    # number of example digits to show
    #n = 5
    j=8000
    plt.figure(figsize=(20, 5))
    for i in range(0, n):
        # plot original image
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(x_test[j].reshape(*dims))
        #plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        if i == n/2:
            ax.set_title('Original Images')

        # plot reconstruction 
        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(x_test_encoded[j].reshape(*dims))
        #plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        if i == n/2:
            ax.set_title('Reconstructed Images')
        j=j+1
    plt.show()

In [ ]:
x_test_encoded = vae.predict(x_test, batch_size = 80)

In [ ]:
print(x_test_encoded[10])

In [ ]:
x_test_encoded

In [ ]:
plot_autoencoder_outputs(x_test_encoded, 10, (34, 68))

In [ ]:
vae_test.set_weights(vae.get_weights())

from random import shuffle

shuffle(x_test)

score = vae_test.evaluate(x_test, x_test, verbose=1, batch_size=80)
#score = vae.evaluate(x_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])